In [18]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [19]:
%sql SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


# Selecting Albums to Purchase

The four most popular genres of tracks in terms of sales in the US are Rock, Alternative & Punk, Metal and R&B/Soul, but each of these are only 3-4% of the total number of sales in the respective genre. The Pop, Blues and Hip-hop genres are the fifth, seventh and ninth most popular genres in terms of US sales. I would therefore recommend that the store purchase the following artists:

* Red Tone
* Slim Jim Bites
* Meteor and the Girls

In [50]:
%%sql 
WITH
country_sales AS
(
    SELECT invoice_id, billing_country
    FROM invoice
),
tracks_sold AS
(
    SELECT track_id, invoice_id, quantity
    FROM invoice_line
),
tracks_genre AS
(
    SELECT t.track_id, t.genre_id, g.name name
    FROM track t
    INNER JOIN genre g ON g.genre_id = t.genre_id 
),
sold_in_USA AS
(
    SELECT tg1.name genre, SUM(ts1.quantity) usa_sales  
    FROM tracks_sold ts1
    INNER JOIN country_sales cs1 ON cs1.invoice_id = ts1.invoice_id
    INNER JOIN tracks_genre tg1 ON tg1.track_id = ts1.track_id   
    WHERE cs1.billing_country = "USA"
    GROUP BY tg1.name
)

SELECT tg.name genre,
       SUM(ts.quantity) "tracks sold",
       s.usa_sales,
       SUM(ts.quantity) / s.usa_sales usa_sales_pct 
FROM tracks_sold ts
INNER JOIN tracks_genre tg ON tg.track_id = ts.track_id
INNER JOIN country_sales cs ON cs.invoice_id = ts.invoice_id
INNER JOIN sold_in_usa s ON s.genre = tg.name
GROUP BY tg.name
ORDER BY s.usa_sales DESC

 * sqlite:///chinook.db
Done.


genre,tracks sold,usa_sales,usa_sales_pct
Rock,2635,561,4
Alternative & Punk,492,130,3
Metal,619,124,4
R&B/Soul,159,53,3
Blues,124,36,3
Alternative,117,35,3
Pop,63,22,2
Latin,167,22,7
Hip Hop/Rap,33,20,1
Jazz,121,14,8


# Analysing Employee Sales Performance

In terms of dollar sales, Jane Peacock has the highest performance with $1,731.51, followed by Margaret Park with $1,584.00 and then Steve Johnson with $1,393.92. The order of performance reflects exactly the order in which each sales support agent joined the company - Jane joined first, so she has more time to accrue sales, compared to Margaret and Steve.  

In [75]:
%%sql
SELECT (e.first_name || " " || e.last_name) employee,
       e.hire_date,
       e.birthdate,
       ROUND(SUM(i.total),2) sales
FROM employee e
LEFT JOIN customer c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id
WHERE e.title = "Sales Support Agent"
GROUP BY employee 
 


 * sqlite:///chinook.db
Done.


employee,hire_date,birthdate,sales
Jane Peacock,2017-04-01 00:00:00,1973-08-29 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1947-09-19 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1965-03-03 00:00:00,1393.92


# Analysing Sales by Country

In [98]:
%%sql

WITH 
avg_order_value_country AS
(
    SELECT c.country, AVG(i.total) avg_total
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    GROUP BY country
)

SELECT c.country,
       COUNT(c.customer_id) "number of customers",
       ROUND(SUM(i.total),2) "total sales",
       CAST(SUM(i.total) / COUNT(c.customer_id) AS INTEGER) "average sales per customer",
       ROUND(aovc.avg_total,2) "Average order value"
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
INNER JOIN avg_order_value_country aovc ON aovc.country = c.country
GROUP BY c.country
ORDER BY "total sales" DESC

 * sqlite:///chinook.db
Done.


country,number of customers,total sales,average sales per customer,Average order value
USA,131,1040.49,7,7.94
Canada,76,535.59,7,7.05
Brazil,61,427.68,7,7.01
France,50,389.07,7,7.78
Germany,41,334.62,8,8.16
Czech Republic,30,273.24,9,9.11
United Kingdom,28,245.52,8,8.77
Portugal,29,185.13,6,6.38
India,21,183.15,8,8.72
Ireland,13,114.84,8,8.83


# Albums v Individual Tracks

Out of a total of 689 invoices, the vast majority (70%) have been for compilations of individual tracks, whilst just 30% have been for album purchases. So, on the face of it, it would appear to make more sense to mainly cater for only individual tracks being added manually to a purchase. Since the store charges the same for an album as it does for each individual track being added manually, there would be no real financial benefit to purchasing whole albums, instead of the individual tracks of an album individually. But, 30% is still a sizeable minority in terms of customer demand (208 out of 689), so only purchasing the most popular tracks from an album could alienate this group. I would therefore recommend that the store continue to buy full albums, and not just the popular tracks from albums.     

In [15]:
%%sql

WITH
invoice_track_category AS
(
    SELECT 
        il.invoice_id,
        t.album_id,
        COUNT(il.track_id) number_of_tracks,
        CASE
            WHEN COUNT(il.track_id) = 1 THEN "No" 
            ELSE "Yes"
        END AS is_album
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id
    GROUP BY il.invoice_id, t.album_id
),
album_purchase AS
(
    SELECT invoice_id, is_album 
    FROM invoice_track_category
    GROUP BY invoice_id, number_of_tracks
)

SELECT COUNT(invoice_id) total_invoices,
       (SELECT COUNT(is_album) FROM album_purchase WHERE is_album = "Yes") yes,
       (SELECT COUNT(is_album) FROM album_purchase WHERE is_album = "No") no,
       ROUND((CAST(COUNT(CASE WHEN is_album = "Yes" THEN 1 ELSE NULL END) AS FLOAT) / COUNT(invoice_id)),2) * 100 pct_yes,
       ROUND((CAST(COUNT(CASE WHEN is_album = "No" THEN 1 ELSE NULL END) AS FLOAT) / COUNT(invoice_id)),2) * 100 pct_no 
FROM album_purchase

 * sqlite:///chinook.db
Done.


total_invoices,yes,no,pct_yes,pct_no
689,208,481,30.0,70.0


# Which artist is used in the largest number of playlists?

Iron Maiden is the artist that appears in the most number of playlists

In [67]:
%%sql

WITH 
artist_album_track AS
(
    SELECT artist.artist_id, artist.name artist, track.track_id track
    FROM artist
    INNER JOIN album ON album.artist_id = artist.artist_id
    INNER JOIN track ON track.album_id = album.album_id
),
track_in_playlist AS
(
    SELECT p.playlist_id, p.name, pt.track_id
    FROM playlist p
    INNER JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
)

SELECT aat.artist, COUNT(tip.name) number_of_playlists
FROM artist_album_track aat
INNER JOIN track_in_playlist tip ON tip.track_id = aat.track
GROUP BY aat.artist
ORDER BY number_of_playlists DESC


 * sqlite:///chinook.db
Done.


artist,number_of_playlists
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Faith No More,145
Eric Clapton,145
Lenny Kravitz,143


# How many tracks have been purchased versus not purchased?

Of the 6,454 tracks available, 4,757 have been purchased and 1,697 have not been purchased.

In [82]:
%%sql

WITH
purchase_status AS
(
    SELECT t.name, il.quantity
    FROM track t
    LEFT JOIN invoice_line il ON il.track_id = t.track_id
)

SELECT COUNT(name) total_tracks,
       COUNT(CASE WHEN quantity > 0 THEN 1 ELSE NULL END) purchased,
       COUNT(name) - COUNT(CASE WHEN quantity > 0 THEN 1 ELSE NULL END) not_purchased 
FROM purchase_status ps

 * sqlite:///chinook.db
Done.


total_tracks,purchased,not_purchased
6454,4757,1697


# Is the range of tracks in the store reflective of their sales popularity?

I interpreted the range of tracks to mean the number of tracks by individual artists. For example, if for each artist, there was equal number of tracks, the range was pretty evenly spread out. On the other hand, there were a lot of artists with a handful of tracks and few of artists with a lot of tracks, then the range would be skewed towards the artists with the most tracks. So, for me, the question was, is the number of tracks by a particular artist reflected in the sales popularity of those tracks? (i.e. Are the most sold tracks by artists with the most number of tracks).

The first stage was to create of table showing the number of tracks in the shop's catalogue per artist.The second stage was to create another table that showed the number of sold tracks per artist. From these two tables, it was possible to identify the artists where the quantity of tracks reflected their sales popularity and the artists where the quantity of the tracks did not reflect their sales popularity.

What I found was that, for the vast majority of the artists, the number of tracks available for sale was not reflected in the number of sales of those tracks.

In [120]:


%%sql

WITH
range_of_tracks AS
(
    SELECT art.name, COUNT(t.track_id) number_tracks
    FROM track t
    INNER JOIN album alb ON alb.album_id = t.album_id
    INNER JOIN artist art ON art.artist_id = alb.artist_id
    GROUP BY art.name
),
track_popularity AS
(
    SELECT art.name, COUNT(il.track_id) number_tracks
    FROM invoice_line il
    INNER JOIN track t ON t.track_id = il.track_id
    INNER JOIN album alb ON alb.album_id = t.album_id
    INNER JOIN artist art ON art.artist_id = alb.artist_id
    GROUP BY art.name
),
range_and_popularity AS
(
    SELECT * FROM range_of_tracks INTERSECT SELECT * FROM track_popularity
),
range_not_popularity AS
(
    SELECT * FROM track_popularity EXCEPT SELECT * FROM range_of_tracks
)


SELECT COUNT(*) FROM range_and_popularity
UNION
SELECT COUNT(*) FROM range_not_popularity

 * sqlite:///chinook.db
Done.


COUNT(*)
7
123
